In [1]:
%matplotlib inline
plt.style.use('ggplot')
import re
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
#import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime
from datetime import timedelta
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.en import English
import pickle

parser = English()
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

from scipy.spatial.distance import pdist


## Import Gachon Corpus into pandas as df

In [3]:
df=pd.read_excel('data/Gachon Learner Corpus.xlsx', error_bad_lines=True)

In [4]:
df.head()

,Timestamp,Please answer the following question. Write between 100 and 150 words. 다음 질문에 100~150단어 답해주십시오.,Student number:,Class number:,Assignment #,Waiver,Gender:,Number of years you have studied English: 영어를 몇년 동안 공부하였습니까?,TOEIC Score 토익 점수,TOEFL Score 토플 점수,...,What year are you in university?,What is your native (first) language?,Father's native (first) language?,Mother's native (first) language?,What languages do you and your family speak in your home?,What language was mostly used in your Elementary School?,What language was mostly used in your Middle School?,What language was mostly used in your High School?,How confident are you in your ability to write in English?,Why are you studying English?
0,2014-03-07 20:13:28,In Korea when people small talk to each other ...,21011008,10075009 월/목 오후1:00,1,1,Male (Man) 남,12,780,x,...,Junior (3학년),Korean,Korean,Korean,Korean,Korean,Korean,Korean,3,Fulfilling Academic Requirements 졸업조건을 충족시키기 위해
1,2014-03-17 05:32:16,I have ever been to austraila in a month. In t...,21011008,10075009 월/목 오후1:00,2,1,Male (Man) 남,12,780,x,...,Junior (3학년),Korean,Korean,Korean,Korean,Korean,Korean,Korean,3,Fulfilling Academic Requirements 졸업조건을 충족시키기 위해
2,2014-03-26 21:48:43,The 'sin sae gye' is the best movie i've ever ...,21011008,10075009 월/목 오후1:00,3,1,Male (Man) 남,12,780,x,...,Junior (3학년),Korean,Korean,Korean,Korean,Korean,Korean,Korean,3,Fulfilling Academic Requirements 졸업조건을 충족시키기 위해
3,2014-04-03 04:42:47,I think if children are allowed to watch viole...,21011008,10075009 월/목 오후1:00,4,1,Male (Man) 남,12,780,x,...,Junior (3학년),Korean,Korean,Korean,Korean,Korean,Korean,Korean,3,Fulfilling Academic Requirements 졸업조건을 충족시키기 위해
4,2014-04-11 22:49:48,"If i am the manager of a hotel, i will have wa...",21011008,10075009 월/목 오후1:00,5,1,Male (Man) 남,12,780,x,...,Junior (3학년),Korean,Korean,Korean,Korean,Korean,Korean,Korean,3,Fulfilling Academic Requirements 졸업조건을 충족시키기 위해


Limit the data set down to pure sentences and tokenize for new pandas file

- Currently in paragraphs
- break it down to to sentences and tokenize

In [5]:
df1 = df.ix[:,1]

df1=df1.drop_duplicates()

In [10]:
text_list=df1.tolist()

In [11]:
print (len(text_list))

25067


### Tokening down to sentence level

In [12]:
sent_list=[]
for item in text_list:
    try:
        sent_tokenize_list = sent_tokenize(item)
        sent_list.append(sent_tokenize_list)   
    except:
        print (item)
    

nan


In [17]:
flattened_sent = [val for sublist in sent_list for val in sublist]

In [20]:
flattened_sent[-1]

'There is no only one answer.'

In [32]:
df2 = pd.DataFrame({'sent': flattened_sent, 'label': 1})

In [33]:
df2.tail()

,label,sent
238574,1,"But the object, money in this case, can be cha..."
238575,1,It can be a life or love or authority or honor...
238576,1,"Moreover, there is difference between thinking..."
238577,1,"If a situation changes, I can change my thinki..."
238578,1,There is no only one answer.


In [34]:
def remove_none_english(item):
    clean_sent=[]
    for word in item:
        if word.isalnum()==True:
            clean_sent.append(word)
    return clean_sent        

In [35]:
def lower_words(item):
    low_words=[]
    for word in item:
        low_words.append(word.lower())
    return low_words    

In [ ]:
df2['sent']=df2['sent'].apply(lambda x: word_tokenize(x))

df2['sent']=df2['sent'].apply(lambda x: remove_none_english(x))

df2['sent']=df2['sent'].apply(lambda x: lower_words(x))

df2['length']=df2['sent'].apply(lambda x: len(x))

df2=df2[df2['length']>1]

In [54]:
df2['sentence']=df2['sent'].apply(lambda x: " ".join(x))

In [55]:
df2.head()

,label,sent,length,sentence
0,1,"[in, korea, when, people, small, talk, to, eac...",17,in korea when people small talk to each other ...
1,1,"[because, they, think, he, or, she, wants, to,...",12,because they think he or she wants to classify...
2,1,"[that, must, be, rude, people, think]",6,that must be rude people think
3,1,"[also, people, do, not, quest, weight, especia...",12,also people do not quest weight especially in ...
4,1,"[korea, women, want, to, hide, her, secret, an...",15,korea women want to hide her secret and man ha...


In [56]:
df2.to_pickle('data/univ_corpus.pkl')

In [53]:
with open('wordvec/univ_kor.pkl', 'wb') as f:
    pickle.dump(lower_word_list, f)